In [73]:
!pip install neo4j
from neo4j import GraphDatabase
import pandas as pd
import psycopg2
from psycopg2 import extras
from psycopg2.extras import execute_values

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
# Define Neo4j connection details
neo4j_uri = "neo4j+s://cff762c5.databases.neo4j.io"
neo4j_user = "neo4j"
neo4j_password = "gpHdU-uQ_wxpqHDsz-UpIwQVKgnBbL6q1jLx1v4g9tA"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [75]:
!curl ipecho.net/plain

35.185.141.235

In [76]:
# Define Postgres connection details
p_host = "34.135.94.149"
p_database = "kamulu2"
p_user = "kamulu2"
p_password = "test"

In [77]:
def neo4j_to_dataframe(neo4j_query):
    # Create a Neo4j driver instance
    driver = GraphDatabase.driver(uri, auth=(username, password))

    # Execute the Neo4j query and create a DataFrame from the results
    with driver.session() as session:
        result = session.run(neo4j_query)
        df = pd.DataFrame([r.values() for r in result], columns=result.keys())

    # Close the Neo4j driver
    driver.close()

    # Return the DataFrame
    return df



In [78]:
def transform_data(df):
    # Convert date fields to datetime objects
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    # Remove null values
    df.dropna(inplace=True)
    
    # Return the transformed DataFrame
    return df



In [79]:


def load_data(df):
    # Connect to Postgres
    conn = psycopg2.connect(host=p_host, database=p_database, user=p_user, password=p_password)
    
    # Create table if it doesn't exist
    with conn.cursor() as cursor:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS telcom_data (
            customer_id INTEGER,
            subscription_id INTEGER,
            service_id INTEGER,
            start_date DATE,
            end_date DATE,
            price FLOAT
        )
        """)
    
    # Load data into table
    with conn.cursor() as cursor:
        # Use the "extras" module to perform a bulk insert
        extras.execute_batch(cursor, """
        INSERT INTO telcom_data (customer_id, subscription_id, service_id, start_date, end_date, price)
        VALUES (%(customer_id)s, %(subscription_id)s, %(service_id)s, %(start_date)s, %(end_date)s, %(price)s)
        """, df.to_dict('records'))
        
    # Commit the transaction and close the connection
    conn.commit()
    conn.close()




In [80]:
import psycopg2
from psycopg2.extras import execute_values

def insert_data(dataframe, conn):
    # Define the SQL query string
    query = """
    INSERT INTO subscriptions (customer_id, subscription_id, service_id, start_date, end_date, price)
    VALUES %s
    """

    # Extract the values from the DataFrame
    values = list(dataframe[['customer_id', 'subscription_id', 'service_id', 'start_date', 'end_date', 'price']].itertuples(index=False, name=None))

    # Use execute_values to insert the values into the database
    with conn.cursor() as cur:
        execute_values(cur, query, values)

    # Commit the transaction and close the connection
    conn.commit()
    


In [81]:
def main():
    # Connect to Neo4j
    driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

    # Extract data from Neo4j
    with driver.session() as session:
        result = session.run(neo4j_query)
        df = pd.DataFrame([r.values() for r in result], columns=result.keys())

    # Transform data
    df = transform_data(df)

    # Connect to PostgreSQL
    conn = psycopg2.connect(host=p_host, database=p_database, user=p_user, password=p_password)

    # Load data into PostgreSQL
    insert_data(df, conn)

    # Close connections
    driver.close()
    conn.close()

if __name__ == "__main__":
    main()
